## assume std is always constant

In [ ]:

# split the data into a train and validation sets 
X1, X2, y1, y2 = train_test_split(X_train, y_train, test_size=0.5) 
# base_model can be any regression 
modelbase_mode.fit(X1, y1)
base_prediction = base_model.predict(X2) 

#compute the RMSE value 
error = mean_squared_error(base_prediction, y2) ** 0.5 
# compute the mean and standard deviation of the distribution 
mean = base_model.predict(X_test) 
st_dev = error

## Fit model to learn error itself

In [ ]:
# split the data in train a validation set 
X1, X2, y1, y2 = train_test_split(X, y, test_size=0.5)
# base_model can be any regression model, a 
# sklearn.ensemble.GradientBoostingRegressor for instance 
base_model.fit(X1, y1) 
base_prediction = base_model.predict(X2) 
# compute the prediction error vector on the validation set
validation_error = (base_prediction - y2) ** 2 
error_model.fit(X2, validation_error) 
# compute the mean and standard deviation of the distribution 
mean = base_model.predict(X_test)
st_dev = error_model.predict(X_test)**0.5

In [ ]:
class QuantileLossFunction(RegressionLossFunction):
    """Loss function for quantile regression.
    Quantile regression allows to estimate the percentiles
    of the conditional distribution of the target.
    Parameters
    ----------
    n_classes : int
        Number of classes.
    alpha : float, optional (default = 0.9)
        The percentile
    """
    def __init__(self, n_classes, alpha=0.9):
        super().__init__(n_classes)
        self.alpha = alpha
        self.percentile = alpha * 100.0

    def init_estimator(self):
        return QuantileEstimator(self.alpha)

    def __call__(self, y, pred, sample_weight=None):
        """Compute the Quantile loss.
        Parameters
        ----------
        y : array, shape (n_samples,)
            True labels
        pred : array, shape (n_samples,)
            Predicted labels
        sample_weight : array-like, shape (n_samples,), optional
            Sample weights.
        """
        pred = pred.ravel()
        diff = y - pred
        alpha = self.alpha

        mask = y > pred
        if sample_weight is None:
            loss = (alpha * diff[mask].sum() -
                    (1.0 - alpha) * diff[~mask].sum()) / y.shape[0]
        else:
            loss = ((alpha * np.sum(sample_weight[mask] * diff[mask]) -
                    (1.0 - alpha) * np.sum(sample_weight[~mask] * diff[~mask])) /
                    sample_weight.sum())
        return loss

    def negative_gradient(self, y, pred, **kargs):
        """Compute the negative gradient.
        Parameters
        ----------
        y : array, shape (n_samples,)
            The target labels.
        pred : array, shape (n_samples,)
            The predictions.
        """
        alpha = self.alpha
        pred = pred.ravel()
        mask = y > pred
        return (alpha * mask) - ((1.0 - alpha) * ~mask)

    def _update_terminal_region(self, tree, terminal_regions, leaf, X, y,
                                residual, pred, sample_weight):
        terminal_region = np.where(terminal_regions == leaf)[0]
        diff = (y.take(terminal_region, axis=0)
                - pred.take(terminal_region, axis=0))
        sample_weight = sample_weight.take(terminal_region, axis=0)

        val = _weighted_percentile(diff, sample_weight, self.percentile)
        tree.value[leaf, 0] = val

